# Do i need volumes join? test on new data, make GUI

In [8]:
from methods import load_data, select_route_data, group_nodes, group_links, obtain_routes, routes_volume_join, drop_duplicates, export_to_json, qgis_json_format, to_list, df_writer
import pathlib
import pandas as pd

In [9]:
file = pathlib.Path.cwd() / "Enfield Saturn Routes AM test.xlsx"

In [10]:
strategic_data_file = pathlib.Path.cwd() / "Enfield Saturn Routes AM test.xlsx"
qgis_file = pathlib.Path.cwd() / "qgis_data.xlsx"
strategic_raw_data, qgis_table = load_data(strategic_data_file, qgis_file) #Load raw data

In [11]:
volumes, nodes = select_route_data(strategic_raw_data)
ogv_volumes, ogv_nodes = select_route_data(strategic_raw_data, ogv=True) #Extract routes for user class 9 (OGVs)

In [12]:
data_sets = {"routes": nodes, "ogv_routes": ogv_nodes}
route_codes=[]
for user_class, data in data_sets.items():
    nodes = to_list(data)
    nodes_grouped=group_nodes(nodes) # Group the node sequences that make up a route
    links = group_links(nodes_grouped) # From the node sequences create the links
    routes = obtain_routes(links, qgis_table) # For the links create the list of links that make up each route
    routes = routes_volume_join(routes) # Associate the volumes to the routes 
    unique_routes = drop_duplicates(routes) # Remove duplicated routes, totalling the volumes (TODO)
    qgis_routes, route_ids = qgis_json_format(unique_routes) if user_class != "ogv_routes" else qgis_json_format(unique_routes, ogv=True) # Format results
    route_codes.append([route_ids[i].split("/")[-1].strip(".gpkg") for i in range(len(route_ids))])
    export_to_json(user_class, qgis_routes) # Export

In [13]:
volumes=volumes.drop_duplicates().dropna()
ogv_volumes=ogv_volumes.drop_duplicates().dropna()

In [14]:
for i, y, name in zip(route_codes, [volumes, ogv_volumes], ["route_volumes", "ogv_route_volumes"]):
    res1= pd.DataFrame(i).reset_index(drop=True)
    res1.columns = ["Origin_Route-ID_UC"]
    res2=pd.DataFrame(y).reset_index(drop=True)
    res3 = pd.concat([res1, res2], axis=1) # SOLVE MISSING DATA POINT
    res3.to_excel(f"{name}.xlsx", index=False)